<a href="https://colab.research.google.com/github/kimiafa/Network/blob/main/crc_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def reflect(value, width):
    """بازتاب بیت‌ها برای حالت LSB-first"""
    result = 0
    for i in range(width):
        if (value >> i) & 1:
            result |= 1 << (width - 1 - i)
    return result


In [2]:
def compute_crc(data, poly, init, xor_out, width, refin=False, refout=False):
    """
    محاسبه CRC با پارامترهای عمومی
    data   : bytes یا str
    poly   : پلی‌نومیال CRC
    init   : مقدار اولیه
    xor_out: مقدار نهایی XOR
    width  : عرض CRC (8,16,32)
    refin  : بازتاب ورودی
    refout : بازتاب خروجی
    """
    if isinstance(data, str):
        data = data.encode()

    mask = (1 << width) - 1
    topbit = 1 << (width - 1)
    crc = init & mask

    if refin:
        poly_ref = reflect(poly, width)
        for b in data:
            crc ^= b
            for _ in range(8):
                if crc & 1:
                    crc = (crc >> 1) ^ poly_ref
                else:
                    crc >>= 1
                crc &= mask
    else:
        for b in data:
            crc ^= (b << (width - 8)) & mask
            for _ in range(8):
                if crc & topbit:
                    crc = ((crc << 1) ^ poly) & mask
                else:
                    crc = (crc << 1) & mask

    if refout:
        crc = reflect(crc, width)

    crc ^= xor_out & mask
    return crc & mask


In [3]:
crc_models = {
    "CRC-8": {
        "poly": 0x07,
        "init": 0x00,
        "xor": 0x00,
        "width": 8
    },
    "CRC-16-IBM": {
        "poly": 0x8005,
        "init": 0xFFFF,
        "xor": 0x0000,
        "width": 16
    },
    "CRC-32": {
        "poly": 0x04C11DB7,
        "init": 0xFFFFFFFF,
        "xor": 0xFFFFFFFF,
        "width": 32
    }
}


In [4]:
data = "Hello CRC"
results = {}

for name, m in crc_models.items():
    results[name] = hex(
        compute_crc(
            data,
            m["poly"],
            m["init"],
            m["xor"],
            m["width"],
            refin=True if name != "CRC-8" else False,
            refout=True if name != "CRC-8" else False
        )
    )

results


{'CRC-8': '0x6a', 'CRC-16-IBM': '0xb640', 'CRC-32': '0x98853d61'}

In [5]:
import random

def introduce_error(data_bytes):
    """تغییر یک بیت در داده شبیه‌سازی خطا"""
    byte_list = list(data_bytes.encode())
    byte_index = random.randint(0, len(byte_list)-1)
    bit_index = random.randint(0,7)
    byte_list[byte_index] ^= (1 << bit_index)
    corrupted = bytes(byte_list)
    return corrupted, (byte_index, bit_index)

corrupted, pos = introduce_error(data)
print("Original:", data)
print("Corrupted:", corrupted)
print("Error position:", pos)


Original: Hello CRC
Corrupted: b'Helmo CRC'
Error position: (3, 0)
